# Big Data & BI – Optional Intro Notebook

**Topic:** Data, EDA


## Learning Objectives
By the end, you can:
- Download a real dataset from the open source projects.
- Evaluate a list of **KPIs**.
- Run simple analysis.


- Open "https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/GTNEJD"
- Download peoplesun_hh_anon.tab as csv (comma seperated valaues).
- Files named peoplesun_hh_odk_codebook.xlsx and peoplesun_hh_odk_choices.xlsx are also needed to decode the questions and answers.
- Place it in your data folder

## Key Performance Indicators (KPIs)

### KPI 1 — Electrification Rate (%)
**Definition:** Percentage of households with any access to electricity (grid or off-grid like solar/generator).  
**Formula:** Electrification Rate = (Households_with_electricity / Total_households) * 100  
**Purpose:** Measures extent of access.

---

### KPI 2 — Supply Reliability (Outages or Hours Without Power)
**Definition:** Average number of power outages per week OR hours without power per day among connected households.  
**Formula (example):** Avg_outages_per_week = SUM(outages_per_week) / N_connected_households  
**Purpose:** Captures quality/stability of supply.

---

### KPI 3 — Energy Burden (%)
**Definition:** Average share of monthly household spending (or income) devoted to electricity expenses.  
**Formula:** Energy_Burden = (Electricity_expenditure / Total_expenditure_or_income) * 100  
**Purpose:** Assesses affordability.


## 1) Setup & load

In [ ]:
# pip install matplotlib

In [4]:
# pip install seaborn

In [2]:
# --- setup ---
import pandas as pd
import numpy as np
import re
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

DATA_PATH = Path("../data/peoplesun_hh_anon.csv")  # students: place file here

# Helpful options
pd.set_option("display.max_colwidth", 120)
pd.set_option("display.max_columns", 120)

# --- load ---
# If the CSV has a different delimiter, change sep="," accordingly.
df = pd.read_csv(DATA_PATH, sep=",", low_memory=False)

print("Rows:", len(df))
df.head()


Rows: 3599


,zone,state,eaid,lga,urca_cat,hhid,q102,q103,q104,q105,q106,q107,q108,q109,q110,q201,q202,q203_1,q203_2,q203_3,q203_4,q204,q205,q206,q207_1,q207_2,q207_3,q208,q209,q210,q210_1,q211,q211__1,q211__2,q211__3,q211__4,q211__5,q211__6,q211__7,q211__8,q211__9,q211__10,q211__11,q211__12,q211__0,q211__96,q211_1,q212,q212_1,q213,q213_1,q213_1__1,q213_1__2,q213_1__96,q214,q214__1,q214__2,q214__3,q214__4,q214__5,...,q402__4,q402__5,q402__6,q404,q404__1,q404__2,q404__3,q404__4,q404__5,q404__6,q404__7,q404__8,q404__9,q404__10,q404__11,q404__0,q404__96,q404_1,q405,q405_1,q405_1__1,q405_1__2,q405_1__3,q405_1__4,q405_1__5,q405_1__6,q405_1__7,q405_1__8,q405_1__9,q405_1__10,q405_1__11,q405_1__12,q405_1__13,q405_1__96,q405_2,q501,q501__1,q501__2,q501__3,q501__4,q501__5,q501__96,q503,q503__1,q503__2,q503__3,q503__4,q503__5,q503__6,q503__99,q503__96,q503__98,q503_1,q604_1,q604_2,q604_3,q604_4,q604_5,q604_6,natweight
0,North Central,Niger,NI_197,mashegu,<1hr to small city/town+,d8af8ab5-30ab-4d9f-bfe4-81231dbe5dbf,1,NaN,1,1,36,2,NaN,NaN,NaN,5,4,13,4,1,0,2,8,6,0,2,0,22,14,96,Politics,2 96,0,1,0,0,0,0,0,0,0,0,0,0,0,1,Politics and farming for consumption,1,1.0,1,1,1.0,0.0,0.0,1 11,1,0,0,0,0,...,1,0,1,2 11,0,1,0,0,0,0,0,0,0,0,1,0,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,0,0,0,1,0,0,1,1,0,0,0,0,0,NaN,0,0,NaN,4,2,3,2,3,3,0.039141
1,North Central,Niger,NI_197,mashegu,<1hr to small city/town+,e8245d5c-8130-4e78-b4b0-1053b7ecbc9b,1,NaN,1,1,60,2,NaN,NaN,NaN,6,2,4,3,2,1,3,9,6,0,0,0,14,13,1,NaN,1 3 96,1,0,1,0,0,0,0,0,0,0,0,0,0,1,Farming for consumption,0,NaN,1,1,1.0,0.0,0.0,11,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,0,0,0,1,0,0,1,1,0,0,0,0,0,NaN,0,0,NaN,4,2,3,1,3,3,0.039141
2,North West,Kano,KN_104,garun_malam,<1hr to large city,435c8e27-517a-46b9-af04-48830e086d7a,1,NaN,1,1,45,1,NaN,NaN,NaN,3,6,3,2,1,0,1,5,6,1,1,0,14,8,2,NaN,2 8,0,1,0,0,0,0,0,1,0,0,0,0,0,0,NaN,1,1.0,1,1,1.0,0.0,0.0,11,0,0,0,0,0,...,1,0,1,11,0,0,0,0,0,0,0,0,0,0,1,0,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,0,0,0,1,0,0,1,1,0,0,0,0,0,0.0,0,0,NaN,4,3,4,2,4,1,0.109779
3,North West,Kano,KN_104,garun_malam,<1hr to large city,9303fa53-9fd2-41a9-9f0d-9567dbe5168e,1,NaN,1,1,47,1,NaN,NaN,NaN,3,15,7,2,1,0,1,10,6,3,1,0,14,8,2,NaN,2 7,0,1,0,0,0,0,1,0,0,0,0,0,0,0,NaN,1,1.0,0,NaN,NaN,NaN,NaN,11,0,0,0,0,0,...,1,1,1,2 11 1,1,1,0,0,0,0,0,0,0,0,1,0,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,0,0,0,1,0,0,1 2,1,1,0,0,0,0,0.0,0,0,NaN,4,3,4,3,4,2,0.109779
4,North West,Kano,KN_104,garun_malam,<1hr to large city,c62cc5a5-29c5-423b-9543-a7b05bda454b,1,NaN,1,1,45,1,NaN,NaN,NaN,2,1,4,1,1,0,2,3,6,0,1,0,24,24,2,NaN,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0,NaN,NaN,NaN,NaN,11,0,0,0,0,0,...,0,0,0,11,0,0,0,0,0,0,0,0,0,0,1,0,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,0,0,0,1,0,0,1 2,1,1,0,0,0,0,0.0,0,0,NaN,4,2,4,2,4,2,0.109779


## 2) Quick EDA (Exploratory Data Analysis)

In [20]:
# --- explore ---
df.head()
df.tail()



,zone,state,eaid,lga,urca_cat,hhid,q102,q103,q104,q105,q106,q107,q108,q109,q110,q201,q202,q203_1,q203_2,q203_3,q203_4,q204,q205,q206,q207_1,q207_2,q207_3,q208,q209,q210,q210_1,q211,q211__1,q211__2,q211__3,q211__4,q211__5,q211__6,q211__7,q211__8,q211__9,q211__10,q211__11,q211__12,q211__0,q211__96,q211_1,q212,q212_1,q213,q213_1,q213_1__1,q213_1__2,q213_1__96,q214,q214__1,q214__2,q214__3,q214__4,q214__5,...,q402__4,q402__5,q402__6,q404,q404__1,q404__2,q404__3,q404__4,q404__5,q404__6,q404__7,q404__8,q404__9,q404__10,q404__11,q404__0,q404__96,q404_1,q405,q405_1,q405_1__1,q405_1__2,q405_1__3,q405_1__4,q405_1__5,q405_1__6,q405_1__7,q405_1__8,q405_1__9,q405_1__10,q405_1__11,q405_1__12,q405_1__13,q405_1__96,q405_2,q501,q501__1,q501__2,q501__3,q501__4,q501__5,q501__96,q503,q503__1,q503__2,q503__3,q503__4,q503__5,q503__6,q503__99,q503__96,q503__98,q503_1,q604_1,q604_2,q604_3,q604_4,q604_5,q604_6,natweight
3594,North Central,Niger,NI_194,chanchaga,<1hr to int. city,9303a8af-9dc3-466f-9fa1-65dbc93c0990,1,0.0,4,1,25,4,1.0,50.0,1.0,6,3,0,4,2,0,1,4,6,0,0,1,22,20,5,0,5 6,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,1.0,0,0,0.0,0.0,0.0,1,1,0,0,0,0,...,1,0,1,2 6 8,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1 4,1,0,0,1,0,0,98,0,0,0,0,0,0,0.0,0,1,0,4,2,4,2,3,2,0.080206
3595,North West,Kaduna,KD_81,kagarko,<1hr to small city/town+,22c88a1a-4e0d-49ec-bfff-8f6f70282e07,1,0.0,1,1,50,1,0.0,0.0,0.0,5,5,2,3,2,0,3,7,6,0,2,0,11,20,7,0,7,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1.0,0,0,0.0,0.0,0.0,1,1,0,0,0,0,...,1,1,1,2 96,0,1,0,0,0,0,0,0,0,0,0,0,1,Stabilizer,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2 4,0,1,0,1,0,0,1 2,1,1,0,0,0,0,0.0,0,0,0,2,2,2,2,3,3,0.069566
3596,North West,Kaduna,KD_82,jaba,<1hr to small city/town+,e3ffc39e-5b54-4de0-857f-d69a9f5de4a9,1,0.0,1,1,42,1,0.0,0.0,0.0,2,4,3,0,2,0,2,4,96,0,1,0,17,14,5,0,1 5,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1.0,1,1,1.0,0.0,0.0,1,1,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5 4,0,0,0,1,1,0,2,0,1,0,0,0,0,0.0,0,0,0,3,2,3,1,3,2,0.069566
3597,North West,Katsina,KT_146,bindawa,Town,d3fb5d54-0e8c-4a67-bbfc-b0ea12ae9438,1,0.0,2,2,35,2,1.0,45.0,2.0,3,8,8,1,2,0,3,11,6,0,1,1,22,20,5,0,5 1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1.0,0,0,0.0,0.0,0.0,1 5 6,1,0,0,0,1,...,1,0,1,2 4 11 7,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1 2 4,1,1,0,1,0,0,1 2,1,1,0,0,0,0,0.0,0,0,0,4,4,4,4,4,4,0.064779
3598,South South,Akwa Ibom,AK_3,uyo,<1hr to large city,c38e98ae-04e5-4419-8dc0-d8174cc0022c,1,0.0,1,1,45,8,0.0,0.0,0.0,7,10,4,3,4,0,2,6,6,0,0,1,20,22,4,0,4 8,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,1.0,1,1,1.0,0.0,0.0,3,0,0,1,0,0,...,1,0,1,2 11,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,0,0,0,0,1,0,1,1,0,0,0,0,0,0.0,0,0,0,3,2,2,2,3,3,0.269222


In [ ]:
df.dtypes
df=df.fillna(0)
df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3599 entries, 0 to 3598
Columns: 293 entries, zone to natweight
dtypes: float64(105), int64(158), object(30)
memory usage: 8.0+ MB


In [13]:
df.head()

,zone,state,eaid,lga,urca_cat,hhid,q102,q103,q104,q105,q106,q107,q108,q109,q110,q201,q202,q203_1,q203_2,q203_3,q203_4,q204,q205,q206,q207_1,q207_2,q207_3,q208,q209,q210,q210_1,q211,q211__1,q211__2,q211__3,q211__4,q211__5,q211__6,q211__7,q211__8,q211__9,q211__10,q211__11,q211__12,q211__0,q211__96,q211_1,q212,q212_1,q213,q213_1,q213_1__1,q213_1__2,q213_1__96,q214,q214__1,q214__2,q214__3,q214__4,q214__5,...,q402__4,q402__5,q402__6,q404,q404__1,q404__2,q404__3,q404__4,q404__5,q404__6,q404__7,q404__8,q404__9,q404__10,q404__11,q404__0,q404__96,q404_1,q405,q405_1,q405_1__1,q405_1__2,q405_1__3,q405_1__4,q405_1__5,q405_1__6,q405_1__7,q405_1__8,q405_1__9,q405_1__10,q405_1__11,q405_1__12,q405_1__13,q405_1__96,q405_2,q501,q501__1,q501__2,q501__3,q501__4,q501__5,q501__96,q503,q503__1,q503__2,q503__3,q503__4,q503__5,q503__6,q503__99,q503__96,q503__98,q503_1,q604_1,q604_2,q604_3,q604_4,q604_5,q604_6,natweight
0,North Central,Niger,NI_197,mashegu,<1hr to small city/town+,d8af8ab5-30ab-4d9f-bfe4-81231dbe5dbf,1,0.0,1,1,36,2,0.0,0.0,0.0,5,4,13,4,1,0,2,8,6,0,2,0,22,14,96,Politics,2 96,0,1,0,0,0,0,0,0,0,0,0,0,0,1,Politics and farming for consumption,1,1.0,1,1,1.0,0.0,0.0,1 11,1,0,0,0,0,...,1,0,1,2 11,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,0,0,0,1,0,0,1,1,0,0,0,0,0,0.0,0,0,0,4,2,3,2,3,3,0.039141
1,North Central,Niger,NI_197,mashegu,<1hr to small city/town+,e8245d5c-8130-4e78-b4b0-1053b7ecbc9b,1,0.0,1,1,60,2,0.0,0.0,0.0,6,2,4,3,2,1,3,9,6,0,0,0,14,13,1,0,1 3 96,1,0,1,0,0,0,0,0,0,0,0,0,0,1,Farming for consumption,0,0.0,1,1,1.0,0.0,0.0,11,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,0,0,0,1,0,0,1,1,0,0,0,0,0,0.0,0,0,0,4,2,3,1,3,3,0.039141
2,North West,Kano,KN_104,garun_malam,<1hr to large city,435c8e27-517a-46b9-af04-48830e086d7a,1,0.0,1,1,45,1,0.0,0.0,0.0,3,6,3,2,1,0,1,5,6,1,1,0,14,8,2,0,2 8,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1.0,1,1,1.0,0.0,0.0,11,0,0,0,0,0,...,1,0,1,11,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,0,0,0,1,0,0,1,1,0,0,0,0,0,0.0,0,0,0,4,3,4,2,4,1,0.109779
3,North West,Kano,KN_104,garun_malam,<1hr to large city,9303fa53-9fd2-41a9-9f0d-9567dbe5168e,1,0.0,1,1,47,1,0.0,0.0,0.0,3,15,7,2,1,0,1,10,6,3,1,0,14,8,2,0,2 7,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1.0,0,0,0.0,0.0,0.0,11,0,0,0,0,0,...,1,1,1,2 11 1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,0,0,0,1,0,0,1 2,1,1,0,0,0,0,0.0,0,0,0,4,3,4,3,4,2,0.109779
4,North West,Kano,KN_104,garun_malam,<1hr to large city,c62cc5a5-29c5-423b-9543-a7b05bda454b,1,0.0,1,1,45,1,0.0,0.0,0.0,2,1,4,1,1,0,2,3,6,0,1,0,24,24,2,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0.0,0.0,0.0,11,0,0,0,0,0,...,0,0,0,11,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,0,0,0,1,0,0,1 2,1,1,0,0,0,0,0.0,0,0,0,4,2,4,2,4,2,0.109779


## 3) KPI 1 — Electrification Rate (%)

In [5]:
# --- explore ---
# Here I have assume q302 question used in the survey for better understanding of the source of electricity used in the various household.

# === 1. Assuming from consumer point of view  ===
# Here I have only consider the options such as National grid connection, Local mini-grid, Solar home system, Solar lantern and Electric generator

# === 2. Identify electricity-related variable ===
# Based on your ODK form, q302 = "What source of electricity is used MOST of the time in this household?"
electricity_var = "q302"

# --- CLEANING & STANDARDIZING RESPONSES ---
# Convert responses to consistent boolean form
# Adjust this mapping based on your codebook (e.g., 1=Yes, 2=No)
df['q302_clean'] = df['q302'].astype(str).str.strip().str.lower()

# === 5. Define what counts as having electricity access  ===
yes_values = ['1', '2', '3', '4']
df['has_electricity'] = df['q302_clean'].isin(yes_values)
print(df['has_electricity'])

# === 4. Compute KPI ===
total_households = df.shape[0]
households_with_electricity = df['has_electricity'].sum()

electrification_rate = (households_with_electricity / total_households) * 100

# === 5. Print Results ===
print(f"Total Households Surveyed: {total_households}")
print(f"Households with Electricity Access: {households_with_electricity}")
print(f"⚡ Electrification Rate (Consumer View): {electrification_rate:.2f}%")

0       False
1       False
2       False
3       False
4       False
        ...  
3594     True
3595    False
3596     True
3597     True
3598    False
Name: has_electricity, Length: 3599, dtype: bool
Total Households Surveyed: 3599
Households with Electricity Access: 2109
⚡ Electrification Rate (Consumer View): 58.60%


In [ ]:
# === 1. Assuming from government point of view  ===
# Here I have only consider the options such as National grid connection and Local mini-grid

# === 2. Define the variable for electricity sources (from your survey)  ===
# Based on your ODK form, q302 = "What source of electricity is used MOST of the time in this household?"
electricity_var = "q302"

# Preview the column to understand its structure
print(df['q302'].value_counts(dropna=False))

# === 3. CLEANING & STANDARDIZING RESPONSES   ====
# Convert responses to consistent boolean form
# Adjust this mapping based on your codebook (e.g., 1=Yes, 2=No)
df['q302_clean'] = df['q302'].astype(str).str.strip().str.lower()

# === 4. Define what counts as having electricity access  ===
yes_values = ['1', '2']
df['has_gov_electricity'] = df['q302_clean'].isin(yes_values)
print(df['has_gov_electricity'])

# === 5. Compute KPI  ===
total_households = len(df)
gov_electrified_households = df['has_gov_electricity'].sum()

gov_electrification_rate = (gov_electrified_households / total_households) * 100

# === 6. Display result  ===
print(f"Total Households Surveyed: {total_households}")
print(f"Households with Electricity Access: {households_with_electricity}")
print(f"🏛️ Electrification Rate (Government View): {gov_electrification_rate:.2f}%")

q302
1     1978
7      664
5      644
6      163
3       71
2       48
96      17
4       12
0        2
Name: count, dtype: int64
0       False
1       False
2       False
3       False
4       False
        ...  
3594     True
3595    False
3596     True
3597     True
3598    False
Name: has_gov_electricity, Length: 3599, dtype: bool
Total Households Surveyed: 3599
Households with Electricity Access: 2109
🏛️ Electrification Rate (Government View): 56.29%


## 4) KPI 2 — Reliability (choose the best available)

In [16]:
# --- explore ---
# For this KPI I have consider National Grid user and Mini Grid user from both Consumer and Government point of view 
# Here I have used below questions from codebook which are specifically asked for National Grid user
# q307_4 → “How many days did you have an unscheduled power outage in the last week?”
# q307_5 → “On average, what was the duration of the outages?”
# q302 → Main electricity source (to confirm “National Grid” users).

# For consumer view we have consider the household connected to the electricty and for Government view we have considrer all the household that the reason we have different output for both.

# KPI 2 for National Grid Users Only

# --- Identify National Grid households (q302) ---
df['q302_clean'] = df['q302'].astype(str).str.strip().str.lower()
grid_values = ['national grid', 'grid', 'nepa', 'phcn']
df['is_grid_user'] = df['q302_clean'].isin(grid_values)

# --- Convert reliability variables for National Grid ---
# q307_4 = number of unscheduled outage days last week
# q307_5 = average outage duration in hours
df['q307_4'] = pd.to_numeric(df['q307_4'], errors='coerce')
df['q307_5'] = pd.to_numeric(df['q307_5'], errors='coerce')

# --- Filter to National Grid users ---
grid_df = df[df['is_grid_user']].copy()

# --- Consumer View (Connected Grid Users) ---
sum_outages_consumer = grid_df['q307_4'].sum()
n_connected_consumer = grid_df['q307_4'].notna().sum()
avg_outages_per_week_consumer = (
    sum_outages_consumer / n_connected_consumer if n_connected_consumer > 0 else 0
)

sum_hours_consumer = grid_df['q307_5'].sum()
n_hours_consumer = grid_df['q307_5'].notna().sum()
avg_outage_hours_consumer = (
    sum_hours_consumer / n_hours_consumer if n_hours_consumer > 0 else 0
)

# --- Government View (All HHs; non-grid = 0) ---
df['q307_4_filled'] = df.apply(lambda x: x['q307_4'] if x['is_grid_user'] else 0, axis=1)
df['q307_5_filled'] = df.apply(lambda x: x['q307_5'] if x['is_grid_user'] else 0, axis=1)

sum_outages_govt = df['q307_4_filled'].sum()
n_households_total = df.shape[0]
avg_outages_per_week_govt = (
    sum_outages_govt / n_households_total if n_households_total > 0 else 0
)

sum_hours_govt = df['q307_5_filled'].sum()
avg_outage_hours_govt = (
    sum_hours_govt / n_households_total if n_households_total > 0 else 0
)

# --- Output summary ---
print("===== KPI 2 — Supply Reliability (National Grid Only) =====")
print("--- Consumer View (Connected HHs) ---")
print(f"Sum of outages/week: {sum_outages_consumer:.2f}")
print(f"Number of connected HHs: {n_connected_consumer}")
print(f"Avg outages/week (Formula): {avg_outages_per_week_consumer:.2f}")
print(f"Avg outage duration (hours): {avg_outage_hours_consumer:.2f}")

print("\n--- Government View (All HHs) ---")
print(f"Sum of outages/week: {sum_outages_govt:.2f}")
print(f"Total households: {n_households_total}")
print(f"Avg outages/week (Formula): {avg_outages_per_week_govt:.2f}")
print(f"Avg outage duration (hours): {avg_outage_hours_govt:.2f}")

===== KPI 2 — Supply Reliability (National Grid Only) =====
--- Consumer View (Connected HHs) ---
Sum of outages/week: 0.00
Number of connected HHs: 0
Avg outages/week (Formula): 0.00
Avg outage duration (hours): 0.00

--- Government View (All HHs) ---
Sum of outages/week: 0.00
Total households: 3599
Avg outages/week (Formula): 0.00
Avg outage duration (hours): 0.00


In [15]:
# Here I have used below questions from codebook which are specifically asked for National Grid user
# q306_4	How many days did you have an unscheduled power outage in the last week?
# q306_5	On average, what was the duration (in hours) of those outages?
# q302	Electricity source (we’ll use this to identify mini-grid users).

# For consumer view we have consider the household connected to the electricty and for Government view we have considrer all the household that the reason we have different output for both.

# KPI 2 for Mini Grid Users Only

# --- Identify Mini Grid households (q302) ---
df['q302_clean'] = df['q302'].astype(str).str.strip().str.lower()
mini_grid_values = ['mini grid', 'community mini grid', 'local mini grid']
df['is_mini_grid_user'] = df['q302_clean'].isin(mini_grid_values)

# --- Convert reliability variables for Mini Grid ---
# q306_4 = number of unscheduled outage days last week
# q306_5 = average outage duration in hours
df['q306_4'] = pd.to_numeric(df['q306_4'], errors='coerce')
df['q306_5'] = pd.to_numeric(df['q306_5'], errors='coerce')

# --- Filter to Mini Grid users ---
mini_df = df[df['is_mini_grid_user']].copy()

# --- Consumer View (connected Mini Grid users only) ---
sum_outages_consumer = mini_df['q306_4'].sum()
n_connected_consumer = mini_df['q306_4'].notna().sum()
avg_outages_per_week_consumer = (
    sum_outages_consumer / n_connected_consumer if n_connected_consumer > 0 else 0
)

sum_hours_consumer = mini_df['q306_5'].sum()
n_hours_consumer = mini_df['q306_5'].notna().sum()
avg_outage_hours_consumer = (
    sum_hours_consumer / n_hours_consumer if n_hours_consumer > 0 else 0
)

# --- Government View (all HHs; non-mini-grid = 0) ---
df['q306_4_filled'] = df.apply(lambda x: x['q306_4'] if x['is_mini_grid_user'] else 0, axis=1)
df['q306_5_filled'] = df.apply(lambda x: x['q306_5'] if x['is_mini_grid_user'] else 0, axis=1)

sum_outages_govt = df['q306_4_filled'].sum()
n_households_total = df.shape[0]
avg_outages_per_week_govt = (
    sum_outages_govt / n_households_total if n_households_total > 0 else 0
)

sum_hours_govt = df['q306_5_filled'].sum()
avg_outage_hours_govt = (
    sum_hours_govt / n_households_total if n_households_total > 0 else 0
)

# --- Output summary ---
print("===== KPI 2 — Supply Reliability (Mini Grid Only) =====")
print("--- Consumer View (Connected HHs) ---")
print(f"Sum of outages/week: {sum_outages_consumer:.2f}")
print(f"Number of connected HHs: {n_connected_consumer}")
print(f"Avg outages/week (Formula): {avg_outages_per_week_consumer:.2f}")
print(f"Avg outage duration (hours): {avg_outage_hours_consumer:.2f}")

print("\n--- Government View (All HHs) ---")
print(f"Sum of outages/week: {sum_outages_govt:.2f}")
print(f"Total households: {n_households_total}")
print(f"Avg outage duration (hours): {avg_outage_hours_govt:.2f}")

===== KPI 2 — Supply Reliability (Mini Grid Only) =====
--- Consumer View (Connected HHs) ---
Sum of outages/week: 0.00
Number of connected HHs: 0
Avg outages/week (Formula): 0.00
Avg outage duration (hours): 0.00

--- Government View (All HHs) ---
Sum of outages/week: 0.00
Total households: 3599
Avg outage duration (hours): 0.00


## 5) KPI 3 — Energy Burden (% of total monthly expenditure or income)

In [ ]:
# --- explore ---
# KPI 3 for National Grid Users Only

# q307_8  monthly electricity expenditure
# q307_10  total monthly household expenditure

# --- Identify National Grid users (q302) ---
df['q302_clean'] = df['q302'].astype(str).str.strip().str.lower()
grid_values = ['national grid', 'grid', 'nepa', 'phcn']
df['is_grid_user'] = df['q302_clean'].isin(grid_values)

# --- Convert expenditure variables ---
# q307_8 = monthly electricity expenditure (National Grid)
# q307_10 = total monthly household expenditure (for National Grid)
df['q307_8'] = pd.to_numeric(df['q307_8'], errors='coerce')
df['q307_10'] = pd.to_numeric(df['q307_10'], errors='coerce')

# --- Filter only National Grid users ---
grid_df = df[df['is_grid_user']].copy()

# --- Calculate Energy Burden (%) using the formula ---
grid_df['energy_burden'] = (grid_df['q307_8'] / grid_df['q307_10']) * 100

# --- Compute averages ---
avg_electricity_expenditure = grid_df['q307_8'].mean()
avg_total_expenditure = grid_df['q307_10'].mean()
avg_energy_burden = grid_df['energy_burden'].mean()

# --- Output summary ---
print("===== KPI 3 — Energy Burden (%) for National Grid Users =====")
print(f"Average monthly electricity expenditure (q307_8): {avg_electricity_expenditure:.2f}")
print(f"Average total monthly household expenditure (q307_10): {avg_total_expenditure:.2f}")
print(f"Average Energy Burden (q307_8 / q307_10 * 100): {avg_energy_burden:.2f}%")

===== KPI 3 — Energy Burden (%) for National Grid Users =====
Average monthly electricity expenditure (q307_8): nan
Average total monthly household expenditure (q307_10): nan
Average Energy Burden (q307_8 / q307_10 * 100): nan%


In [29]:
# --- explore ---
# KPI 3 for Mini Grid Users Only

# q307_8  monthly electricity expenditure
# q307_10  total monthly household expenditure

# --- Identify Mini Grid users (q302) ---
df['q302_clean'] = df['q302'].astype(str).str.strip().str.lower()
mini_grid_values = ['mini grid', 'community mini grid', 'local mini grid']
df['is_mini_grid_user'] = df['q302_clean'].isin(mini_grid_values)

# --- Convert expenditure variables ---
# q306_8 = monthly electricity expenditure (for Mini Grid)
# q306_10 = total monthly household expenditure (for Mini Grid)
df['q306_8'] = pd.to_numeric(df['q306_8'], errors='coerce')
df['q306_10'] = pd.to_numeric(df['q306_10'], errors='coerce')

# --- Filter only Mini Grid users ---
mini_df = df[df['is_mini_grid_user']].copy()

# --- Calculate Energy Burden (%) using the formula ---
mini_df['energy_burden'] = (mini_df['q306_8'] / mini_df['q306_10']) * 100

# --- Compute averages ---
avg_electricity_expenditure = mini_df['q306_8'].mean()
avg_total_expenditure = mini_df['q306_10'].mean()
avg_energy_burden = mini_df['energy_burden'].mean()

# --- Output summary ---
print("===== KPI 3 — Energy Burden (%) for Mini Grid Users =====")
print(f"Average monthly electricity expenditure (q306_8): {avg_electricity_expenditure:.2f}")
print(f"Average total monthly household expenditure (q306_10): {avg_total_expenditure:.2f}")
print(f"Average Energy Burden (q306_8 / q306_10 * 100): {avg_energy_burden:.2f}%")

===== KPI 3 — Energy Burden (%) for Mini Grid Users =====
Average monthly electricity expenditure (q306_8): nan
Average total monthly household expenditure (q306_10): nan
Average Energy Burden (q306_8 / q306_10 * 100): nan%
